In [47]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [48]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [21]:
# import required libraries
import os
import sys
import glob
import numpy as np
import pandas as pd

import SimpleITK as sitk
import nibabel as nib
import pydicom

import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

In [15]:
# all required functions
def sanity_check_volume_slices(dir_path):
    file_paths= glob.glob(dir_path + '*.nii')
    volume_data = []
    
    for fullname in file_paths:
        nii_obj = nib.load(fullname)
        (nx, ny, nz) = nii_obj.shape[0:3]
        (sx, sy, sz) = nii_obj.header.get_zooms()    #  Slice thickness
        voxel_spacing = sx * sy* sz
        filename = fullname.split(os.sep)[-1]
        data = [filename, nx, ny, nz, voxel_spacing]
        volume_data.append(data)

    return volume_data

In [16]:
data_dir= '/kaggle/input/cynch-seg-seg/'
print(os.listdir(data_dir))

img_dir= os.path.join(data_dir, 'images/')
mask_vessel= os.path.join(data_dir, 'human_corrected_with_vessel_removal/')
mask_novessel= os.path.join(data_dir, 'human_corrected_without_vessel_removal/')


img_files = sorted(glob.glob(os.path.join(img_dir, '*.nii')))
mask_vessel_files = sorted(glob.glob(os.path.join(mask_vessel, '*.nii')))
mask_novessel_files = sorted(glob.glob(os.path.join(mask_novessel, '*.nii')))


print(f'Total # of images: {len(img_files)}' )
print(f' Total # of masks, no vessel: {len(mask_novessel_files)}')
print(f'Total #  of masks, with vessel: {len(mask_vessel_files)}')

In [18]:
volume_data = sanity_check_volume_slices(img_dir)
df = pd.DataFrame(volume_data, columns= ['volume', 'width', 'height', '#_slices', 'voxel_spacing'])
# print(df)

In [32]:
nii_obj = nib.load(img_files[4])
(nx, ny, nz) = nii_obj.shape[0:3]

img_data = nib.load(img_files[4]).get_fdata()
mask_data = nib.load(mask_vessel_files[4]).get_fdata()

In [22]:
img_slice = img_data[:, :, 17]
mask_slice = mask_data[:, :, 17]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
ax1.imshow(img_slice, cmap= 'bone')
ax1.set_title('Image')

ax2.imshow(mask_slice, cmap= 'bone')
ax2.set_title('Mask with vessels')

In [30]:
def plot_sample(input_img, input_mask, color_map = 'nipy_spectral'):
    '''
    Plots and a slice with all available annotations
    '''
    fig = plt.figure(figsize=(20,16), dpi=100)

    plt.subplot(1,3,1)
    plt.imshow(img_slice, cmap='bone')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1,3,2)
    plt.imshow(mask_slice, alpha=0.5, cmap=color_map)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1,3,3)
    plt.imshow(img_slice, cmap='bone')
    plt.imshow(mask_slice, alpha=0.5, cmap=color_map)
    plt.title('Liver & Mask')
    plt.axis('off')
    
    plt.show()

In [31]:
plot_sample(img_slice, mask_slice)

In [37]:
for i in range(nz):
    f = plt.figure(figsize = (15,15))
    f.add_subplot(1,2,1)
    plt.imshow(img_data[:,:,i], cmap='bone')
    f.add_subplot(1,2,2)
    plt.imshow(mask_data[:,:,i], cmap='nipy_spectral')

In [39]:
for i in range(nz):
    f = plt.figure(figsize = (5,5))
    plt.imshow(mask_data[:,:,i], cmap='nipy_spectral')

In [46]:
print('Done!')